In [2]:
import gc
import importlib

import torch
import sys
from pathlib import Path
module_path = str(Path.cwd().parents[0] / "network_models/soundstream_models_and_utils/encoder")
if module_path not in sys.path:
    sys.path.append(module_path)

device = "cpu"

import network_models.soundsream_models_and_utils.encoder.ss_encoder_downmapping as ed
importlib.reload(ed)
from network_models.soundsream_models_and_utils.encoder.transformer import Encoder

#encoder = Encoder(vocab_size=1024, embed_dim=512, n_heads=1, ff_dim=0, n_layers=2, dropout=0)
encoder = ed.EncoderDownmapping(embed_dim=512, n_heads=4, ff_dim=2, n_layers=1, dropout=0, output=1024, max_seq_len=175).to(device)

gc.collect()

8

In [3]:
from network_models.soundsream_models_and_utils.ss_encoded_dataset import ss_encoded_dataset_full

data_set= ss_encoded_dataset_full(
    csvPath="/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/data/allEncodings.pkl", device="cuda")

2023-02-16 12:02:46.824484: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-16 12:02:47.491311: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-16 12:02:47.491373: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-02-16 12:02:47.491378: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [ ]:
# import torch
# data_set[0][0][0].T[0]

In [4]:
import sys
from pathlib import Path

module_path = str(Path.cwd().parents[0] / "network_models/soundstream_models_and_utils/encoder")
if module_path not in sys.path:
    sys.path.append(module_path)

torch.unsqueeze(data_set[0][0][0].T,0).shape

batch = torch.stack([ data_set[1][0][0], data_set[0][0][0], data_set[2][0][0], data_set[3][0][0]])

In [3]:
# batch = torch.stack([ data_set[1][0][0].T, data_set[0][0][0].T, data_set[2][0][0].T, data_set[3][0][0].T])


In [4]:
batch = torch.transpose(batch, 1,2)

In [29]:
batch.shape

torch.Size([4, 175, 512])

In [33]:
batch_emo = [data_set[0][1], data_set[1][1], data_set[2][1], data_set[3][1]]
batch.shape

torch.Size([4, 175, 512])

In [5]:
with torch.no_grad():
    encoder = encoder.to("cpu")
    x = encoder(batch)

with torch.no_grad():
    encoder1 = encoder.to("cuda")
    batch1 = batch.to("cuda")
    x_cuda = encoder1(batch1)
x.shape

torch.Size([4, 1024])

In [8]:
print(batch)
print(batch1)

tensor([[[-2.0999e+00,  1.5010e+00, -5.8182e-01,  ...,  1.0235e-01,
           1.0431e+00,  1.3878e+00],
         [-1.2212e+00, -8.2663e-02,  2.2457e-02,  ...,  3.1075e-01,
           2.2923e-01,  8.4323e-01],
         [-1.0300e+00, -3.5000e-01,  6.2621e-01,  ...,  1.0394e+00,
          -3.0595e-01,  1.7874e-01],
         ...,
         [-6.2423e+00,  1.8420e+00, -7.6485e-01,  ..., -4.3404e-01,
           1.6344e+00,  1.8389e+00],
         [-6.1685e+00,  1.9278e+00, -7.4513e-01,  ..., -4.5462e-01,
           1.6922e+00,  1.8381e+00],
         [-6.2048e+00,  1.9243e+00, -7.4410e-01,  ..., -4.4282e-01,
           1.6600e+00,  1.8191e+00]],

        [[-2.3367e+00,  1.6229e+00, -2.9810e-01,  ...,  2.0379e-01,
           1.0990e+00,  1.3267e+00],
         [-1.6375e+00,  7.5411e-02,  3.3088e-01,  ...,  5.0821e-01,
           2.5376e-01,  9.4094e-01],
         [-1.3727e+00, -2.5153e-01,  9.4419e-01,  ...,  1.1065e+00,
          -3.5122e-01,  1.3705e-01],
         ...,
         [-2.5371e+00,  1

In [6]:
import numpy as np
import torch.nn.functional as F
mul = torch.matmul(x, x.T)
F.softmax(mul/np.sqrt(torch.sum(torch.matmul(x, x.T), dim=0)), dim=0)



mul_cuda = torch.matmul(x_cuda, x_cuda.T)

print(mul, mul_cuda)

# x1=torch.dot(x[0],x[0])
# x2=torch.dot(x[0],x[1])
# x3=torch.dot(x[0],x[2])
# x4=torch.dot(x[0],x[3])
# calcLoss=[]
# for elem in x:
#     calcLoss.append(torch.dot(elem, x[0]))
#     calcLoss.append(torch.dot(elem, x[1]))
#     calcLoss.append(torch.dot(elem, x[2]))
#     calcLoss.append(torch.dot(elem, x[3]))
# t = torch.reshape(torch.asarray(calcLoss), (4,4))
# #torch.softmax(t[0]/340, dim=0)
# t
# #F.softmax(t[0])

tensor([[345.9051, 261.1588, 311.9456, 299.9271],
        [261.1588, 335.4135, 261.3019, 260.6291],
        [311.9456, 261.3019, 341.6087, 302.7792],
        [299.9271, 260.6291, 302.7792, 337.2212]]) tensor([[345.9051, 261.1589, 311.9456, 299.9271],
        [261.1589, 335.4135, 261.3019, 260.6291],
        [311.9456, 261.3019, 341.6087, 302.7791],
        [299.9271, 260.6291, 302.7791, 337.2211]], device='cuda:0')


In [2]:
import torch.nn.functional as F
import torch
import numpy as np


/tmp/ipykernel_932512/2714952727.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  F.softmax(torch.tensor(np.asarray([-20, 0.1, 0.2])))


tensor([8.8592e-10, 4.7502e-01, 5.2498e-01], dtype=torch.float64)

In [ ]:
encoder.parameters

In [53]:
np.identity(4)

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [8]:
mul = torch.matmul(x, x.T)
mul

tensor([[352.3570, 285.2893, 318.3849, 310.8500],
        [285.2893, 360.8303, 283.4177, 289.1945],
        [318.3849, 283.4177, 347.4364, 307.3763],
        [310.8500, 289.1945, 307.3763, 356.1323]])

In [7]:
F.softmax(torch.matmul(x, x.T)/ torch.sum(torch.matmul(x, x.T), dim=0))

NameError: name 'F' is not defined

In [86]:
torch.sum(torch.matmul(x, x.T), dim=0)

tensor([ 998.2489, 1117.8684, 1119.7424, 1106.6016])

In [10]:
max = torch.max(mul, dim=0)[0]
F.softmax(mul/np.sqrt(torch.sum(torch.matmul(x, x.T), dim=0)), dim=0)

tensor([[0.5410, 0.0850, 0.2286, 0.1660],
        [0.0822, 0.7395, 0.0852, 0.0902],
        [0.2083, 0.0805, 0.5187, 0.1505],
        [0.1685, 0.0950, 0.1675, 0.5933]])

In [120]:
F.softmax(mul/np.sqrt(1024), dim=0)

tensor([[0.7136, 0.0195, 0.1759, 0.1184],
        [0.0166, 0.9355, 0.0178, 0.0249],
        [0.1697, 0.0202, 0.6930, 0.1295],
        [0.1000, 0.0247, 0.1134, 0.7272]])